<a href="https://colab.research.google.com/github/oribar1/MVP_Prediction/blob/main/MVP_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
stats = pd.read_csv("player_mvp_stats.csv")

In [ ]:
del stats["Unnamed: 0"]

In [ ]:
pd.isnull(stats).sum()

In [ ]:
stats = stats.fillna(0)

In [ ]:
stats.columns

In [ ]:
predictors = ['Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year',
        'W', 'L', 'W/L%', 'GB', 'PS/G',
       'PA/G', 'SRS']

In [ ]:
train = stats[stats["Year"]<2021]

In [ ]:
test = stats[stats["Year"] == 2021]

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
reg = Ridge(alpha=.1)

In [ ]:
reg.fit(train[predictors],train["Share"])

Ridge(alpha=0.1)

In [ ]:
predictions = reg.predict(test[predictors])

In [ ]:
predictions = pd.DataFrame(predictions,columns= ["predictions"],index=test.index)

In [ ]:
predictions

,predictions
630,0.013567
631,-0.013756
632,0.002414
633,-0.004421
634,0.010734
...,...
13897,-0.012571
13898,-0.011575
13899,0.016424
13900,-0.020434


In [ ]:
combine = pd.concat([test[["Player","Share"]],predictions],axis=1)

In [ ]:
combine.sort_values("predictions",ascending=False)

,Player,Share,predictions
9907,Giannis Antetokounmpo,0.345,0.207436
8624,Joel Embiid,0.580,0.162713
641,Nikola Jokić,0.961,0.154307
10997,Luka Dončić,0.042,0.151430
3736,LeBron James,0.001,0.147511
...,...,...,...
9437,Patrick McCaw,0.000,-0.050344
13476,Killian Hayes,0.000,-0.051316
9915,P.J. Tucker,0.000,-0.054995
3862,Anžejs Pasečņiks,0.000,-0.062399


In [ ]:
from sklearn.metrics import mean_squared_error

mean_squared_error(combine["Share"],combine["predictions"])

0.0026668954567104115

In [ ]:
combine = combine.sort_values("Share",ascending=False)

In [ ]:
combine["Rk"] = list(range(1,combine.shape[0]+1))

In [ ]:
combine = combine.sort_values("predictions",ascending=False)
combine["Predicted_Rk"] = list(range(1,combine.shape[0]+1))

In [ ]:
def find_ap(combine):
    actual = combine.sort_values("Share", ascending=False).head(5)
    predicted = combine.sort_values("predictions", ascending=False)
    ps = []
    found = 0
    seen = 1
    for index,row in predicted.iterrows():
        if row["Player"] in actual["Player"].values:
            found += 1
            ps.append(found / seen)
        seen += 1

    return sum(ps) / len(ps)

In [ ]:
years = list(range(1991,2022))

In [ ]:
aps = []
all_predictions = []
for year in years[5:]:
    train = stats[stats["Year"] < year]
    test = stats[stats["Year"] == year]
    reg.fit(train[predictors],train["Share"])
    predictions = reg.predict(test[predictors])
    predictions = pd.DataFrame(predictions, columns=["predictions"], index=test.index)
    combine = pd.concat([test[["Player", "Share"]], predictions], axis=1)
    all_predictions.append(combine)
    aps.append(find_ap(combine))

In [ ]:
sum(aps)/len(aps)

0.7112884360789578

In [ ]:
def add_ranks(predictions):
    predictions = predictions.sort_values("predictions", ascending=False)
    predictions["Predicted_Rk"] = list(range(1,predictions.shape[0]+1))
    predictions = predictions.sort_values("Share", ascending=False)
    predictions["Rk"] = list(range(1,predictions.shape[0]+1))
    predictions["Diff"] = (predictions["Rk"] - predictions["Predicted_Rk"])
    return predictions

In [ ]:
add_ranks(all_predictions[1])

,Player,Share,predictions,Predicted_Rk,Rk,Diff
1600,Karl Malone,0.857,0.192318,2,1,-1
10524,Michael Jordan,0.832,0.167629,3,2,-1
908,Grant Hill,0.327,0.128646,6,3,-3
4682,Tim Hardaway,0.207,0.059984,20,4,-16
8248,Glen Rice,0.117,0.033110,53,5,-48
...,...,...,...,...,...,...
10136,Horacio Llamas,0.000,0.010171,156,437,281
3576,Ennis Whatley,0.000,0.010250,155,438,283
10594,Kevin Salvadori,0.000,0.010553,154,439,285
1138,Aaron Williams,0.000,0.010594,153,440,287


In [ ]:
def backtest(stats, model, years, predictors):
    aps = []
    all_predictions = []
    for year in years:
        train = stats[stats["Year"] < year]
        test = stats[stats["Year"] == year]
        model.fit(train[predictors],train["Share"])
        predictions = model.predict(test[predictors])
        predictions = pd.DataFrame(predictions, columns=["predictions"], index=test.index)
        combine = pd.concat([test[["Player", "Share"]], predictions], axis=1)
        combine = add_ranks(combine)
        all_predictions.append(combine)
        aps.append(find_ap(combine))
    return sum(aps) / len(aps), aps, pd.concat(all_predictions)

In [ ]:
mean_ap, aps, all_predictions = backtest(stats, reg, years[5:], predictors)

In [ ]:
mean_ap

0.7112884360789578

In [ ]:
all_predictions[all_predictions["Rk"] < 5].sort_values("Diff").head(10)

,Player,Share,predictions,Predicted_Rk,Rk,Diff
1224,Jason Kidd,0.712,0.028209,52,2,-50
5175,Steve Nash,0.839,0.034100,45,1,-44
8516,Peja Stojaković,0.228,0.036269,38,4,-34
12726,Joakim Noah,0.258,0.046968,37,4,-33
5193,Steve Nash,0.739,0.054128,34,1,-33
5208,Steve Nash,0.785,0.074421,21,2,-19
4682,Tim Hardaway,0.207,0.059984,20,4,-16
937,Allen Iverson,0.270,0.072120,14,4,-10
6746,Kobe Bryant,0.291,0.078014,14,4,-10
1157,Gary Payton,0.372,0.076708,13,3,-10


In [ ]:
stat_ratios = stats[["PTS", "AST", "STL", "BLK", "3P", "Year"]].groupby("Year").apply(lambda x: x/x.mean())

,PTS,AST,STL,BLK,3P,Year
0,1.013334,0.420714,0.961127,0.673469,0.508587,1.0
1,1.614653,1.028412,1.647646,0.673469,4.577279,1.0
2,0.311795,0.093492,0.274608,1.571429,0.000000,1.0
3,0.200440,0.186984,0.274608,0.000000,0.000000,1.0
4,2.383005,1.636110,1.784950,0.897959,1.525760,1.0
...,...,...,...,...,...,...
14087,0.735752,0.819562,0.479763,1.528302,0.650951,1.0
14088,0.071202,0.000000,0.000000,0.000000,0.130190,1.0
14089,1.281633,0.601012,1.119447,2.547170,0.520761,1.0
14090,0.474679,0.218550,0.319842,1.273585,0.650951,1.0


In [ ]:
stats[["PTS_R", "AST_R", "STL_R", "BLK_R", "3P_R"]] = stat_ratios[["PTS", "AST", "STL", "BLK", "3P"]]

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,W/L%,GB,PS/G,PA/G,SRS,PTS_R,AST_R,STL_R,BLK_R,3P_R
0,A.C. Green,PF,27,LAL,82,21,26.4,3.1,6.6,0.476,...,0.707,5.0,106.3,99.6,6.73,1.013334,0.420714,0.961127,0.673469,0.508587
1,Byron Scott,SG,29,LAL,82,82,32.1,6.1,12.8,0.477,...,0.707,5.0,106.3,99.6,6.73,1.614653,1.028412,1.647646,0.673469,4.577279
2,Elden Campbell,PF,22,LAL,52,0,7.3,1.1,2.4,0.455,...,0.707,5.0,106.3,99.6,6.73,0.311795,0.093492,0.274608,1.571429,0.000000
3,Irving Thomas,PF,25,LAL,26,0,4.2,0.7,1.9,0.340,...,0.707,5.0,106.3,99.6,6.73,0.200440,0.186984,0.274608,0.000000,0.000000
4,James Worthy,SF,29,LAL,78,74,38.6,9.2,18.7,0.492,...,0.707,5.0,106.3,99.6,6.73,2.383005,1.636110,1.784950,0.897959,1.525760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14087,Spencer Hawes,PF,28,MIL,54,1,14.8,2.5,5.1,0.484,...,0.512,9.0,103.6,103.8,-0.45,0.735752,0.819562,0.479763,1.528302,0.650951
14088,Steve Novak,PF,33,MIL,8,0,2.8,0.3,0.9,0.286,...,0.512,9.0,103.6,103.8,-0.45,0.071202,0.000000,0.000000,0.000000,0.130190
14089,Terrence Jones,PF,25,MIL,54,12,23.5,4.3,9.1,0.470,...,0.512,9.0,103.6,103.8,-0.45,1.281633,0.601012,1.119447,2.547170,0.520761
14090,Thon Maker,C,19,MIL,57,34,9.9,1.5,3.2,0.459,...,0.512,9.0,103.6,103.8,-0.45,0.474679,0.218550,0.319842,1.273585,0.650951


In [ ]:
predictors += ["PTS_R", "AST_R", "STL_R", "BLK_R", "3P_R"]

In [ ]:
mean_ap, aps, all_predictions = backtest(stats, reg, years[5:], predictors)

In [ ]:
mean_ap

0.7208380973034985

In [ ]:
stats["NPos"] = stats["Pos"].astype("category").cat.codes
stats["NTm"] = stats["Tm"].astype("category").cat.codes

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=50, random_state=1, min_samples_split=5)

mean_ap, aps, all_predictions = backtest(stats, rf, years[28:], predictors + ["NPos", "NTm"])

0.836417678182384

In [ ]:
mean_ap, aps, all_predictions = backtest(stats, reg, years[28:], predictors)

0.7981818181818182